In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from mlxtend.preprocessing import standardize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from keras import optimizers
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

Using TensorFlow backend.


### Read in Dataset and do initial evaluation

In [69]:
df = pd.read_csv('/Users/dwade/Desktop/NCHS_-_Leading_Causes_of_Death__United_States.csv', header = None)
df = df.rename(columns = df.iloc[0]) #Rename columns appropriately
df = df.drop([0]) #Remove row containing index

In [70]:
df.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
1,2016,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2755,55.50
2,2016,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,439,63.10
3,2016,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4010,54.20
4,2016,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1604,51.80
5,2016,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,California,13213,32.00


In [71]:
df.shape
# 6 variables with 10 thousand records

(10296, 6)

In [72]:
df.isna().sum()
#No missing variables

Year                       0
113 Cause Name             0
Cause Name                 0
State                      0
Deaths                     0
Age-adjusted Death Rate    0
dtype: int64

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10296 entries, 1 to 10296
Data columns (total 6 columns):
Year                       10296 non-null object
113 Cause Name             10296 non-null object
Cause Name                 10296 non-null object
State                      10296 non-null object
Deaths                     10296 non-null object
Age-adjusted Death Rate    10296 non-null object
dtypes: object(6)
memory usage: 563.1+ KB


### Reduce memory and recast variables

In [74]:
print('Total size of dataset:', df.memory_usage().sum()/1000, 'Kb') 
#Initially 576.576Kb

Total size of dataset: 576.576 Kb


In [93]:
df['Deaths'] = pd.to_numeric(df['Deaths'])
df['Year'] = pd.to_numeric(df['Year'])
df['Age-adjusted Death Rate'] = pd.to_numeric(df['Age-adjusted Death Rate'])
df['State'] = df['State'].astype('category')
df['113 Cause Name'] = df['113 Cause Name'].astype('category')
df['Cause Name'] = df['Cause Name'].astype('category')

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10296 entries, 1 to 10296
Data columns (total 6 columns):
Year                       10296 non-null int16
113 Cause Name             10296 non-null category
Cause Name                 10296 non-null category
State                      10296 non-null category
Deaths                     10296 non-null int32
Age-adjusted Death Rate    10296 non-null float32
dtypes: category(3), float32(1), int16(1), int32(1)
memory usage: 214.9 KB


In [95]:
df['Deaths'] = pd.to_numeric(df['Deaths'], downcast = 'integer')
df['Year'] = pd.to_numeric(df['Year'], downcast = 'integer')
df['Age-adjusted Death Rate'] = pd.to_numeric(df['Age-adjusted Death Rate'], downcast = 'float')

In [96]:
print('Current size of dataset:', df.memory_usage().sum()/1000, 'Kb') 
#Now 220Kb

Current size of dataset: 220.008 Kb


### Report the range of each variable

In [105]:
minMax1 = {'max': [1,2], 'min':[3,4]} #temporary dataframe, will override 
Min = pd.DataFrame(data = minMax1)

def minMax(x): #Function to creat dataframe of min's and max's
    Min[x.name] = [x.max(), x.min()]

temp = df.drop(columns = 'Cause Name')
temp = temp.drop(columns = '113 Cause Name')
temp = temp.drop(columns = 'State')

temp.apply(minMax, axis = 0)
Min = Min.drop(columns = 'max')
Min = Min.drop(columns = 'min')
Min = Min.rename(index={0:' Max'})
Min = Min.rename(index={1:' Min'})
Min

,Year,Deaths,Age-adjusted Death Rate
Max,2016.0,2712630.0,1087.300049
Min,1999.0,21.0,2.600000


In [110]:
temp.head()

,Year,Deaths,Age-adjusted Death Rate
1,2016,2755,55.500000
2,2016,439,63.099998
3,2016,4010,54.200001
4,2016,1604,51.799999
5,2016,13213,32.000000


### Initial EDA Plots

In [ ]:
fig = plt.figure(figsize = (10,10))
fig.subplots_adjust(hspace = 0.9, wspace = 0.5)
for i in range(1, len(temp.columns)): #Drop target to not plot target against itself
    fig.add_subplot(5,3,i)
    sns.boxplot(x = 'Age-adjusted Death Rate', y = temp.columns[i], data = temp)
plt.show()

#.drop('Age-adjusted Death Rate')